**Install Packages**

In [1]:
!sudo apt-get install -y jq
!pip install uv
!uv pip install paginate-json --system

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  libjq1 libonig5
The following NEW packages will be installed:
  jq libjq1 libonig5
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 357 kB of archives.
After this operation, 1,087 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy/main amd64 libonig5 amd64 6.9.7.1-2build1 [172 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 libjq1 amd64 1.6-2.1ubuntu3 [133 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 jq amd64 1.6-2.1ubuntu3 [52.5 kB]
Fetched 357 kB in 0s (2,355 kB/s)
debconf: unable to initialize frontend: Dialog
debconf: (No usable dialog-like program is installed, so the dialog based frontend cannot be used. at /usr/share/perl5/Debconf/FrontEnd/Dialog.pm line 78, <> line 3.)
debconf: falling back to frontend: Readline
debconf: unable to initializ

**Download Dataset**

In [2]:
from google.colab import userdata
with open("GITHUB_API_KEY.txt", "w") as f:
    f.write(userdata.get('GITHUB_API_KEY'))

In [3]:
![[ ! -f bazel-closed-issues.txt ]] && paginate-json 'https://api.github.com/repos/bazelbuild/bazel/issues?state=closed&filter=all' --header Authorization "bearer $(cat GITHUB_API_KEY.txt)" \
  | jq '[.[] | {id: .id, title: .title}]' | tee bazel-closed-issues.txt

In [4]:
!cat bazel-closed-issues.txt | jq '[.[] | .title]' | tee bazel-closed-issue-titles.txt

Streaming output truncated to the last 5000 lines.
  "partial downloads can cause confusing error messages.",
  "Explicitly assign --buildroot in the rpmbuild invocation",
  "remote/http: fully read http content on error status",
  "Fix typo in `install-ubuntu.md`",
  "Ubuntu 18.04 / Miniconda: Empty trust anchor",
  "[feature request] support unix domain sockets for the REST remote cache",
  "AddFiles will recursively add files when a directory is encountered",
  "chown in fakeroot fails in Bazel sandbox",
  "DON'T MERGE -- Run CI tests only",
  "DON'T MERGE -- running tests only",
  "Provide support for tree artifacts with all languages ",
  "Tree artifact as input for cc_library fails with headers",
  "Corrupt installation on Windows",
  "bazel uses invalid options when building with gcc on OS X",
  "Migrate pkg_rpm to rules_pkg",
  "Support completion for fish shell",
  "Windows: Building py binaries remotely is broken (\"rule is missing dependency declarations\")",
  "Skylark API 

In [91]:
!cat bazel-closed-issue-titles.txt | wc -l

21464


**Train Autoencoder**

In [5]:
from sentence_transformers import SentenceTransformer

embedder = SentenceTransformer("all-MiniLM-L6-v2")

/usr/local/lib/python3.10/dist-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [6]:
with open("bazel-closed-issue-titles.txt") as f:
    corpus = f.readlines()

corpus_embeddings = embedder.encode(corpus)

In [45]:
from sentence_transformers import util
clusters = util.community_detection(corpus_embeddings, min_community_size=3, threshold=0.70)

In [46]:
# Print for all clusters the top 3 and bottom 3 elements
for i, cluster in enumerate(clusters):
    print(f"\nCluster {i + 1}, #{len(cluster)} Elements ")
    for sentence_id in cluster[0:3]:
        print("\t", corpus[sentence_id])
    print("\t", "...")
    for sentence_id in cluster[-3:]:
        print("\t", corpus[sentence_id])

Streaming output truncated to the last 5000 lines.

	   "[7.2.0] Create remoteOutputService within beforeCommand method",

	   "Create remoteOutputService within beforeCommand method",


Cluster 1567, #3 Elements 
	   "[7.2.0] cpp/blaze_util: handle clock_gettime failure",

	   "[7.2.0] cpp/blaze_util: handle clock_gettime failure",

	   "cpp/blaze_util: handle clock_gettime failure",

	 ...
	   "[7.2.0] cpp/blaze_util: handle clock_gettime failure",

	   "[7.2.0] cpp/blaze_util: handle clock_gettime failure",

	   "cpp/blaze_util: handle clock_gettime failure",


Cluster 1568, #3 Elements 
	   "JDK 8's tools.jar not collected by DumpPlatformClassPath.java",

	   "DumpPlatformClassPath regression with --host_javabase=<jdk8>",

	   "DumpPlatformClassPath.java (part of Java toolchain) is non-deterministic",

	 ...
	   "JDK 8's tools.jar not collected by DumpPlatformClassPath.java",

	   "DumpPlatformClassPath regression with --host_javabase=<jdk8>",

	   "DumpPlatformClassPath.java (part

**Clustering Quality Metrics**

In [47]:
from sklearn.metrics.pairwise import cosine_distances

distance_matrix = cosine_distances(corpus_embeddings)

In [48]:
cluster_labels = [-1] * len(corpus_embeddings)  # Initialize with -1 (unassigned)
for cluster_id, cluster in enumerate(clusters):
    for sentence_idx in cluster:
        cluster_labels[sentence_idx] = cluster_id

cluster_labels

[-1,
 10,
 10,
 1473,
 -1,
 -1,
 -1,
 -1,
 537,
 18,
 1474,
 0,
 912,
 913,
 1474,
 914,
 914,
 -1,
 -1,
 1474,
 18,
 -1,
 915,
 86,
 -1,
 0,
 -1,
 849,
 -1,
 86,
 86,
 915,
 -1,
 1473,
 1473,
 86,
 0,
 -1,
 913,
 914,
 -1,
 -1,
 912,
 912,
 912,
 -1,
 1037,
 849,
 -1,
 -1,
 -1,
 -1,
 1475,
 396,
 1475,
 1475,
 916,
 916,
 30,
 -1,
 179,
 849,
 63,
 1476,
 1152,
 1152,
 -1,
 28,
 -1,
 537,
 1195,
 -1,
 396,
 181,
 185,
 1477,
 913,
 913,
 396,
 86,
 86,
 63,
 14,
 -1,
 206,
 86,
 -1,
 -1,
 1476,
 35,
 -1,
 0,
 0,
 19,
 86,
 15,
 35,
 1477,
 1477,
 916,
 0,
 0,
 -1,
 917,
 -1,
 -1,
 -1,
 1277,
 1277,
 1478,
 86,
 -1,
 916,
 607,
 0,
 917,
 917,
 1078,
 1190,
 1479,
 1480,
 918,
 1481,
 0,
 307,
 378,
 -1,
 -1,
 1478,
 1478,
 1481,
 915,
 915,
 537,
 -1,
 1481,
 49,
 -1,
 -1,
 1190,
 1476,
 -1,
 1479,
 159,
 0,
 0,
 500,
 309,
 10,
 325,
 150,
 257,
 -1,
 14,
 15,
 1480,
 -1,
 918,
 86,
 86,
 1479,
 15,
 10,
 -1,
 -1,
 918,
 1482,
 0,
 80,
 547,
 547,
 547,
 -1,
 325,
 20,
 20,
 325,
 -1

In [49]:
len(cluster_labels)

21464

In [50]:
cluster_labels.count(-1)

8659

In [71]:
from sklearn.metrics import silhouette_score

score = silhouette_score(distance_matrix, cluster_labels, metric='precomputed')
print("Silhouette Score:", score)

Silhouette Score: -0.074097194


In [84]:
clustered_indexes = []

for idx, label in enumerate(cluster_labels):
  if label != -1: # unclustered
    clustered_indexes.append(idx)

In [78]:
clustered_distance_matrix = distance_matrix[clustered_indexes, :][:, clustered_indexes]

In [79]:
len(clustered_distance_matrix)

12805

In [81]:
only_clustered_labels = [label for label in cluster_labels if label != -1]

In [82]:
from sklearn.metrics import silhouette_score

score = silhouette_score(clustered_distance_matrix, only_clustered_labels, metric='precomputed')
print("Silhouette Score:", score)

Silhouette Score: 0.18378456


In [86]:
from sentence_transformers import util
clusters = util.community_detection(corpus_embeddings, min_community_size=15, threshold=0.85)

In [89]:
distance_matrix = cosine_distances(corpus_embeddings)

cluster_labels = [-1] * len(corpus_embeddings)  # Initialize with -1 (unassigned)
for cluster_id, cluster in enumerate(clusters):
    for sentence_idx in cluster:
        cluster_labels[sentence_idx] = cluster_id

clustered_indexes = []
for idx, label in enumerate(cluster_labels):
  if label != -1: # unclustered
    clustered_indexes.append(idx)
print(len(clustered_indexes))

clustered_distance_matrix = distance_matrix[clustered_indexes, :][:, clustered_indexes]
only_clustered_labels = [label for label in cluster_labels if label != -1]

score = silhouette_score(clustered_distance_matrix, only_clustered_labels, metric='precomputed')
print("Silhouette Score:", score)

862
Silhouette Score: 0.46831518
